In [ ]:
# # Debugging ONLY! autoreloads when scripts update
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import contextily as ctx

import os

from src import preprocessing, config

# Obtain raw data

In [2]:
# Get 2020 Decennial Census data
decennial2020_dp_raw = preprocessing.fetch_2020_demographic_profile()

GET request status: 200


In [3]:
np.array(decennial2020_dp_raw).shape

(2328, 646)

# Perform initial cleaning

In [3]:
# Clean NYC census tract shapefiles
gdf = preprocessing.clean_tracts(config.TRACTS_RAW, config.TRACTS_CLEAN)

gdf.sample(n=5)

,BOROUGH,TRACT,AREA,LAT,LONG,geometry
GEOID,,,,,,
36005006200,The Bronx,62,194004,40.833288,-73.875519,"POLYGON ((-73.87925 40.83442, -73.87841 40.835..."
36047005602,Brooklyn,56.02,135056,40.612738,-74.035739,"POLYGON ((-74.04645 40.61073, -74.03903 40.613..."
36081073700,Queens,737,210609,40.718440,-73.843249,"POLYGON ((-73.84623 40.71787, -73.8459 40.7184..."
36047030800,Brooklyn,308,681579,40.586551,-73.979511,"POLYGON ((-73.98742 40.58513, -73.98729 40.585..."
36061014601,Manhattan,146.01,74344,40.777192,-73.952161,"POLYGON ((-73.95448 40.77814, -73.95398 40.778..."


In [2]:
decennial2020_dp_clean = preprocessing.clean_2020_demographic_profile()

decennial2020_dp_clean.sample(n=5)

,pop,pop_under5,pop_5to9,pop_10to14,pop_15to19,pop_20to24,pop_25to29,pop_30to34,pop_35to39,pop_40to44,pop_45to49,pop_50to54,pop_55to59,pop_60to64,pop_65to69,pop_70to74,pop_75to79,pop_80to84,pop_85plus,pop_16plus,pop_18plus,pop_21plus,pop_62plus,pop_65plus,pop_male,pop_male_under5,pop_male_5to9,pop_male_10to14,pop_male_15to19,pop_male_20to24,pop_male_25to29,pop_male_30to34,pop_male_35to39,pop_male_40to44,pop_male_45to49,pop_male_50to54,pop_male_55to59,pop_male_60to64,pop_male_65to69,pop_male_70to74,pop_male_75to79,pop_male_80to84,pop_male_85plus,pop_male_16plus,pop_male_18plus,pop_male_21plus,pop_male_62plus,pop_male_65plus,pop_female,pop_female_under5,pop_female_5to9,pop_female_10to14,pop_female_15to19,pop_female_20to24,pop_female_25to29,pop_female_30to34,pop_female_35to39,pop_female_40to44,pop_female_45to49,pop_female_50to54,pop_female_55to59,pop_female_60to64,pop_female_65to69,pop_female_70to74,pop_female_75to79,pop_female_80to84,pop_female_85plus,pop_female_16plus,pop_female_18plus,pop_female_21plus,pop_female_62plus,pop_female_65plus,medianage,medianage_male,medianage_female,pop_onerace,pop_white,pop_black,pop_nativeam,pop_asian,pop_hipacific,pop_other,pop_multi,racetally,racetally_white,racetally_black,racetally_nativeam,racetally_asian,racetally_hipacific,racetally_other,pop_hispanic,pop_nonhispanic,pop_hispanic_white,pop_hispanic_black,pop_hispanic_nativeam,pop_hispanic_asian,pop_hispanic_hipacific,pop_hispanic_other,pop_hispanic_multi,pop_nonhispanic_white,pop_nonhispanic_black,pop_nonhispanic_nativeam,pop_nonhispanic_asian,pop_nonhispanic_hipacific,pop_nonhispanic_other,pop_nonhispanic_multi,pop_household,pop_household_householder,pop_household_spouse_oppsex,pop_household_spouse_samesex,pop_household_unmpartner_oppsex,pop_household_unmpartner_samesex,pop_household_child,pop_household_child_under18,pop_household_grandchild,pop_household_grandchild_under18,pop_household_otherrelatives,pop_household_nonrelatives,pop_gpquarters,pop_gpquarters_institutionalized,pop_gpquarters_institutionalized_male,pop_gpquarters_institutionalized_female,pop_gpquarters_noninst,pop_gpquarters_noninst_male,pop_gpquarters_noninst_female,households,households_married,households_married_withchild,households_cohabit,households_cohabit_withchild,households_singlemale,households_singlemale_alone,households_singlemale_alone_65plus,households_singlemale_withchild,households_singlefemale,households_singlefemale_alone,households_singlefemale_alone_65plus,households_singlefemale_withchild,households_withunder18,households_with65plus,housing,housing_occupied,housing_vacant,housing_vacant_forrent,housing_vacant_rentedunocc,housing_vacant_forsale,housing_vacant_soldunocc,housing_vacant_seasonal,housing_vacant_other,vacancyrate_homeowner,vacancyrate_rental,housing_owneroccupied,housing_renteroccupied
GEOID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36081059300,4092,206,171,191,190,266,382,378,341,272,259,238,278,275,236,152,105,66,86,3497,3414,3304,815,645,2047,119,82,99,105,134,199,182,176,122,140,118,127,145,124,80,43,24,28,1729,1689,1630,391,299,2045,87,89,92,85,132,183,196,165,150,119,120,151,130,112,72,62,42,58,1768,1725,1674,424,346,38.7,37.5,39.8,3793,2718,99,21,280,1,674,299,4423,2947,182,60,331,10,893,1041,3051,155,21,19,7,0,641,198,2563,78,2,273,1,33,101,4092,1608,634,14,116,11,1050,603,68,44,349,242,0,0,0,0,0,0,0,1619,648,247,118,33,392,196,57,40,461,240,107,51,421,471,1755,1619,136,52,2,5,0,4,73,1.1,4.2,442,1177
36081035300,2828,175,187,169,180,203,269,219,240,191,204,160,188,138,96,81,46,52,30,2266,2191,2082,383,305,1387,85,99,79,87,81,132,124,129,95,101,83,96,60,53,35,21,20,7,1112,1077,1019,169,136,1441,90,88,90,93,122,137,95,111,96,103,77,92,78,43,46,25,32,23,1154,1114,1063,214,169,35.3,35.3,35.3,2472,212,328,96,270,0,1566,356,3199,471,396,128,288,10,1906,2103,725,127,31,84,2,0,1530,329,85,297,12,268,0,36,27,2828,806,332,3,65,1,847,5